# LeRobot v0.4.0

## 概要

[LeRobot v0.4.0][1]の新機能を試す

[1]: https://github.com/huggingface/lerobot/releases/tag/v0.4.0

- Dataset v3の導入
    - 1エピソード1ファイルから、複数のエピソードを単一のParquet/MP4ファイルに集約する形式に変更
    - エピソードの参照にリレーショナルメタデータを使用
    - StreamingLeRobotDatasetによりデータセットのストリームでのダウンロードが可能になった
    - v2.1データセットをv3に変換するスクリプトを追加 
- 新しいポリシーの追加
    - NVIDIA GR00TN 1.5を追加
    - LiberoとMetaWorldのシミュレーション環境を追加
    - Reachy 2ロボットとIntel XPUバックエンドへの対応を追加
- パフォーマンスの向上
    - accelerateライブラリを用いたマルチGPUトレーニングに対応
    - データ拡張機能にアフィン変換を追加
    - APIの拡張（build_inference_frame・make_robot_action）
- コードベースの大幅なリファクタリング
    - RL・非同期処理関連の実装がトップレベルに移動
    - mypyによる型チェックを導入
    - 古いコンポーネントを非推奨化

## 検証設定

In [ ]:
%load_ext autoreload
%autoreload 2

## インストール

[Install LeRobot][1]

[1]: https://huggingface.co/docs/lerobot/en/installation#install-lerobot-

In [ ]:
!python --version # 3.10
!ffmpeg -version # 7.X

In [ ]:
import os

if not os.path.exists("lerobot"):
    !git clone --branch v0.4.0-1-gc75455a6 --depth 1 https://github.com/huggingface/lerobot.git

%cd lerobot
%pip install -qe . # LeRobotの依存関係をインストール
%pip show lerobot # 0.4.1

## Dataset v3

### エピソードベースからファイルベースへ

Dataset v2.1は、ファイルシステムの制限により数百万ものエピソードに対応できなかった

Dataset v3では、複数のエピソードをまとめたファイルベースに変更され、より大きなファイルで管理するようになった:

![](image/dataset.png)

### データセットの構成

- テーブルデータ
    - 状態・アクション・タイムスタンプ
    - Apache Parquet形式
- 視覚データ
    - カメラフレームは連結されMP4にエンコード
    - 同一エピソードでグループ化
    - シャーディングはカメラごとに行う
- メタデータ
    - フレームレート・統計量・共有Parquet/MP4ファイルへのエピソードの開始終了オフセット 
    - JSON・Parquet形式

### データセットのロード

In [ ]:
%pip install -U huggingface-hub==0.35.3 traitlets==5.14.3

In [ ]:
%pip show tqdm

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="defunct-datasets/amazon_reviews_multi", repo_type="dataset")

In [ ]:
%pip install -U tqdm


In [ ]:
import torch
from lerobot.datasets.lerobot_dataset import LeRobotDataset, CODEBASE_VERSION

repo_id = "yaak-ai/L2D-v3"

# 1) Load from the Hub (cached locally)
dataset = LeRobotDataset(repo_id, force_cache_sync=True)
dataset

In [ ]:
# 2) Random access by index
sample = dataset[100]
print(sample)
# {
#   'observation.state': tensor([...]),
#   'action': tensor([...]),
#   'observation.images.front_left': tensor([C, H, W]),
#   'timestamp': tensor(1.234),
#   ...
# }

# 3) Temporal windows via delta_timestamps (seconds relative to t)
delta_timestamps = {
    "observation.images.front_left": [-0.2, -0.1, 0.0]  # 0.2s and 0.1s before current frame
}

dataset = LeRobotDataset(repo_id, delta_timestamps=delta_timestamps)

# Accessing an index now returns a stack for the specified key(s)
sample = dataset[100]
print(sample["observation.images.front_left"].shape)  # [T, C, H, W], where T=3

# 4) Wrap with a DataLoader for training
batch_size = 16
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

device = "cuda" if torch.cuda.is_available() else "cpu"
for batch in data_loader:
    observations = batch["observation.state"].to(device)
    actions = batch["action"].to(device)
    images = batch["observation.images.front_left"].to(device)
    # model.forward(batch)

### ディレクトリレイアウト

- 